In [1]:
from pitchtypes import AbstractBase, Spelled, Enharmonic, LogFreq
from math import log

## Basic Pitch Types

Both _SpelledPitch_ and _EnharmonicPitch_ can be initialised from commonly used pitch notation. However, _SpelledPitch_ will (you guessed it) retain the spelling while _EnharmonicPitch_ converts to the corresponding enharmonically equivalent pitch (internally stored as the corresponding integer in MIDI representation). When printing 'black keys', _EnharmonicPitch_ will need to add an accidental, which by default is a sharp (the 'flat' version can be obtained by explicitly calling the _name_ function with the corresponding argument). _SpelledPitch_ can be converted to _EnharmonicPitch_ but not the other way around.

In [2]:
for pitch in ["F##", "Bbb", "C#", "Db"]:
    print(f"Spelled('{pitch}'):\t\t{Spelled(pitch)}")
    p = Enharmonic(pitch)
    print(f"Enharmonic('{pitch}'):\t{p}\t[{p.name(sharp_flat='sharp')}/{p.name(sharp_flat='flat')}]")
    print(f"{Spelled(pitch)} --> {Enharmonic(Spelled(pitch))}")
    # print(f"{Enharmonic(pitch)} --> {Spelled(Enharmonic(pitch))}")  # NotImplementedError
    print()

Spelled('F##'):		F##
Enharmonic('F##'):	G	[G/G]
F## --> G

Spelled('Bbb'):		Bbb
Enharmonic('Bbb'):	A	[A/A]
Bbb --> A

Spelled('C#'):		C#
Enharmonic('C#'):	C#	[C#/Db]
C# --> C#

Spelled('Db'):		Db
Enharmonic('Db'):	C#	[C#/Db]
Db --> C#



### Pitch versus Pitch Classe

In fact, the example above is using pitch _classes_, that is the octave is ignored. If you want actual pitches, you will need to specify the octave.

In [3]:
for pitch in ["Dbb1", "C2", "B#3", "C4"]:
    midi = int(Enharmonic(pitch))
    print(f"{Spelled(pitch)}\t{Enharmonic(pitch)}\t(MIDI: {midi} <--> {Enharmonic(midi, is_pitch=True, is_class=False)})")

Dbb1	C1	(MIDI: 24 <--> C1)
C2	C2	(MIDI: 36 <--> C2)
B#3	C4	(MIDI: 60 <--> C4)
C4	C4	(MIDI: 60 <--> C4)


As you can see, _EnharmonicPitch_ can also be converted back and forth from/to the corresponding MIDI integer representation (when providing an integer you have to assist by specifying how it should be interpreted). And if you have a pitch (or interval) object, you can easily convert it to the corresponding pitch (or interval) _class_:

In [4]:
p = Enharmonic("C4")
i = Enharmonic(17, is_pitch=False, is_class=False)
pc = p.to_class()
ic = i.to_class()
print(f"{p} --> {pc}")
print(f"{i} --> {ic}")

C4 --> C
+17 --> +5


### Frequencies

If you are interested in the frequency corresponding to a specific pitch...well, you should think twice what that means, but here are some ways to deal with frequencies.

We use the _LogFreqPitch_ type to represent frequencies. As the name suggests, it corresponds to the logarithm of the frequency (in Hertz) and is internally represented as a float:

In [5]:
print(LogFreq(log(440.1)))
print(LogFreq(440.1, is_freq=True))
print(f"{float(LogFreq(log(440.1)))} == {log(440.1)}")

440.1Hz
440.1Hz
6.0870019738170456 == 6.0870019738170456


For other pitch types, it is not obvious how they relate to frequency, because this generally depends on the specific tuning being used. To still be as user-friendly as possible, we assume that people using _EnharmonicPitch_ will generally use twelve-tone equal temperament, so we provide a default conversion to frequency:

In [6]:
print(Enharmonic("A4").freq())
print(LogFreq(Enharmonic("A4")))

440.0
440.Hz


In contrast, _SpelledPitch_ pitch contains additional information that may be used to adjust the frequency in an appropriate way, depending on the tuning and/or the musical context. So we do not provide a default conversion and the following would result in an _NotImplementedError_:

In [7]:
# LogFreq(Spelled("A4"))  # NotImplementedError

You can still perform the conversion explicitly, in which case we trust you that you know what you are doing:

In [8]:
LogFreq(Enharmonic(Spelled("A4")))

440.Hz

## Intervals and Arithmetics

ToDo...

In [9]:
p1 = Spelled("C#4")
p2 = Spelled("Gb5")
i = p2 - p1
print(f"{p2} - {p1} = {i}")
print(f"{p2} + {i} = {p2 + i}")
print(f"{p2} - {i} = {p2 - i}")
print(f"{i} - {i} = {i - i}")
print(f"{i} + {i} = {i + i}")
# p1 + p2                                   # TypeError (cannot add pitches)
# Spelled("G") - Spelled("G4")    # TypeError (cannot mix pitches with pitch classes)
# Spelled("G") - Enharmonic("C")  # TypeError (cannot mix different pitch types)

Gb5 - C#4 = +dd5:1
Gb5 + +dd5:1 = Dbbb6
Gb5 - +dd5:1 = C#4
+dd5:1 - +dd5:1 = +P1:0
+dd5:1 + +dd5:1 = +ddd2:2


## Converters

When implementing new pitch types, you can register converters to allow (implicit) conversion to and from existing pitch types.

In [10]:
class TypeA(AbstractBase): pass
class TypeB(AbstractBase): pass
class TypeC(AbstractBase): pass

AbstractBase.register_converter(from_type=TypeA, 
                         to_type=TypeB, 
                         conv_func=lambda pitch_a: TypeB(pitch_a._value, pitch_a.is_pitch(), pitch_a.is_class()))
AbstractBase.register_converter(from_type=TypeB, 
                         to_type=TypeC, 
                         conv_func=lambda pitch_b: TypeC(pitch_b._value, pitch_b.is_pitch(), pitch_b.is_class()))

print(TypeA("foo", True, False).convert_to(TypeB))
print(TypeB("bar", True, False).convert_to(TypeC))

TypeBPitch(foo)
TypeCPitch(bar)


### Implicit Converters

In [11]:
print(TypeA("baz", True, False).convert_to(TypeC))  # What?!

TypeCPitch(baz)


We did not provide a converter from TypeA to TypeC, instead, an _implicit_ converter was automatically added by chaining the other two: TypeA --> TypeB --> TypeC. This can be supressed as follows (here for the inverse conversion):

In [12]:
AbstractBase.register_converter(from_type=TypeC,
                         to_type=TypeB, 
                         conv_func=lambda pitch_c: TypeB(pitch_c._value, pitch_c.is_pitch(), pitch_c.is_class()))
AbstractBase.register_converter(from_type=TypeB,
                         to_type=TypeA, 
                         conv_func=lambda pitch_b: TypeA(pitch_b._value, pitch_b.is_pitch(), pitch_b.is_class()),
                         create_implicit_converters=False)  # Here, the implicit converter would have been created!
print(TypeC("foo", True, False).convert_to(TypeB))
print(TypeB("bar", True, False).convert_to(TypeA))
# print(TypeC("baz", True, False).convert_to(TypeA))  # NotImplementedError

TypeBPitch(foo)
TypeAPitch(bar)


You can look at all registered converters and see how the implicit converter from above is made up from two separate conversion steps arranged into a pipeline. In fact, there are only the two predefined converters we already know (_Spelled_ --> _Enharmonic_, _Enharmonic_ --> _LogFreq_) and those we just defined (including the implicitly defined one):

In [13]:
for from_type, d in AbstractBase._converters.items():
    print(f"from: {from_type}")
    for to_type, f in d.items():
        print(f"    to: {to_type}:\t conversion pipeline: {f}")

from: <class 'pitchtypes.datatypes.Spelled'>
    to: <class 'pitchtypes.datatypes.Enharmonic'>:	 conversion pipeline: [<function Enharmonic.convert_from_Spelled at 0x10dbe6040>]
from: <class 'pitchtypes.datatypes.Enharmonic'>
    to: <class 'pitchtypes.datatypes.LogFreq'>:	 conversion pipeline: [<function LogFreq.convert_from_midi_pitch at 0x10dbe65e0>]
from: <class '__main__.TypeA'>
    to: <class '__main__.TypeB'>:	 conversion pipeline: [<function <lambda> at 0x10dbe6dc0>]
    to: <class '__main__.TypeC'>:	 conversion pipeline: [<function <lambda> at 0x10dbe6dc0>, <function <lambda> at 0x10dbe6f70>]
from: <class '__main__.TypeB'>
    to: <class '__main__.TypeC'>:	 conversion pipeline: [<function <lambda> at 0x10dbe6f70>]
    to: <class '__main__.TypeA'>:	 conversion pipeline: [<function <lambda> at 0x10dbe6e50>]
from: <class '__main__.TypeC'>
    to: <class '__main__.TypeB'>:	 conversion pipeline: [<function <lambda> at 0x10dbe6af0>]
